In [1]:
import numpy as np
import pandas as pd

# 8.1 层次化索引

In [2]:
# 层次化索引(hierarchical indexing): 使你能在一个轴上拥有多个（两个以上）索引级别，即：使你能以低维度形式处理高维度数据
# 创建一个Series,并用一个由列表或数组组成的列表作为索引

data = pd.Series(np.random.randn(9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                      [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [3]:
# 结果是经过美化的带有MultiIndex索引的Series格式
# 索引之间的“间隔”表示“直接使用上面的标签”

data

a  1   -0.773383
   2    0.914824
   3    1.136187
b  1   -0.651662
   3    1.760072
c  1   -1.333617
   2    0.688691
d  2   -1.266600
   3   -0.252280
dtype: float64

In [4]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [5]:
# 对于一个层次化索引对象，可以使用【部分索引】，以使选取数据子集的操作更简单

data['b']

1    0.041923
3   -1.630202
dtype: float64

In [6]:
data['b': 'c']

b  1    0.041923
   3   -1.630202
c  1   -0.884635
   2   -0.317644
dtype: float64

In [7]:
data.loc[['b', 'd']]

b  1    0.041923
   3   -1.630202
d  2   -0.763775
   3   -1.197224
dtype: float64

In [8]:
# 在【内层】中进行选取

data.loc[ : , 2]

a   -0.850557
c   -0.317644
d   -0.763775
dtype: float64

In [4]:
# 层次化索引在【数据重塑】和【基于分组的操作（如透视表生成）】中扮演着重要的角色
# instance.unstack(): 将这段数据重新安排到一个DataFrame中

dt = data.unstack()
dt

1         2         3
a -0.773383  0.914824  1.136187
b -0.651662       NaN  1.760072
c -1.333617  0.688691       NaN
d       NaN -1.266600 -0.252280

In [5]:
# instance.stack(): 是其逆运算

dt.stack()

a  1   -0.773383
   2    0.914824
   3    1.136187
b  1   -0.651662
   3    1.760072
c  1   -1.333617
   2    0.688691
d  2   -1.266600
   3   -0.252280
dtype: float64

In [6]:
# 对于一个DataFrame,每条【轴】都可以有分层索引

frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [7]:
# 每【层】都可以有名字
# 如果指定名称，则会显示在控制台输出中

frame.index.names = ['key1', 'key2']  # index 行
frame.columns.names = ['state', 'color']  # columns 列

In [8]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [9]:
# 有了部分【列索引】，可以轻松选取【列分组】

frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [10]:
# 可以单独创建MultiIndex，然后复用
# 上面那个DataFrame中的（带有分级名称）列可以这样创建

# MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
#                        names=['state','color'])

## 8.1.1 重排与分级排序

In [22]:
import numpy as np
import pandas as pd

In [23]:
# 有时，需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序

frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame.index.names = ['key1', 'key2']  # index 行
frame.columns.names = ['state', 'color']  # columns 列
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [24]:
# instance.swaplevel(): 接受两个级别【编号】或【名称】，并返回一个互换了级别的新对象（但数据不会发生变化）

# frame.swaplevel('key1', 'key2')  # 交换 key1 与 key2 的顺序
frame.swaplevel(0, 1)  # 等价

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [30]:
# instance.sort_index(): 根据单个级别中的值对数据进行排序。

frame.sort_index(level=1)



state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [31]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## 8.1.2 根据级别汇总统计

In [32]:
import numpy as np
import pandas as pd

In [33]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame.index.names = ['key1', 'key2']  # index 行
frame.columns.names = ['state', 'color']  # columns 列
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [34]:
# 许多对DataFrame和Series的描述和汇总统计都有一个【level选项】：用于指定在某条轴上【求和】的级别
# 这其实是利用了pandas的groupby功能

frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [35]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## 8.1.3 使用DataFrame的列进行索引

In [36]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two','two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

a  b    c  d
0  0  7  one  0
1  1  6  one  1
2  2  5  one  2
3  3  4  two  0
4  4  3  two  1
5  5  2  two  2
6  6  1  two  3

In [44]:
# DataFrame_instance.set_index(): 将其一个或多个列转换为行索引，并创建一个新的DataFrame

frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [46]:
# 默认情况下，那些列会从DataFrame中【移除】， 但也可以将其【保留】下来: drop = False

frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [47]:
# instance.reset_index(): 与set_index相反，层次化索引的级别会被转移到【列】里面

frame2.reset_index()

c  d  a  b
0  one  0  0  7
1  one  1  1  6
2  one  2  2  5
3  two  0  3  4
4  two  1  4  3
5  two  2  5  2
6  two  3  6  1

# 8.2 合并数据集

In [2]:
import numpy as np
import pandas as pd

## 8.2.1 数据库风格的DataFrame合并

In [49]:
# 数据集的合并(merge)或连接(join)运算是通过一个或多个键将行连接起来的

df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                   'data1': range(7)})
df1

key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6

In [57]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                   'data2': range(6, 9)})

df2

key  data2
0   a      6
1   b      7
2   d      8

In [58]:
# pd.merge(): 多对一的合并

pd.merge(df1, df2)

key  data1  data2
0   b      0      7
1   b      1      7
2   b      6      7
3   a      2      6
4   a      4      6
5   a      5      6

In [59]:
# 注意：该例并没有指明要用哪个列进行连接。
# 如果没有指定，merge就会将重叠列的列名当做键
# 不过，最好明确指定一下。

pd.merge(df1, df2, on='key')

key  data1  data2
0   b      0      7
1   b      1      7
2   b      6      7
3   a      2      6
4   a      4      6
5   a      5      6

In [60]:
# 如果两个对象的列名不同，也可以分别进行指定

df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df3

lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6

In [61]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})

df4

rkey  data2
0    a      0
1    b      1
2    d      2

In [62]:
# 其中，c和d以及与之相关的数据消失了。
# 默认情况下，merge做的是【内连接】：结果中的键是【交集】。

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

lkey  data1 rkey  data2
0    b      0    b      1
1    b      1    b      1
2    b      6    b      1
3    a      2    a      0
4    a      4    a      0
5    a      5    a      0

In [63]:
# 其他方式还有：'left', 'right', 'outer',等。【外连接】：求取的是键的【并集】

pd.merge(df1, df2, how='outer')

key  data1  data2
0   b    0.0    7.0
1   b    1.0    7.0
2   b    6.0    7.0
3   a    2.0    6.0
4   a    4.0    6.0
5   a    5.0    6.0
6   c    3.0    NaN
7   d    NaN    8.0

In [68]:
%%html
<img src="picture/表8-1 不同的连接类型.png", width=600>

In [77]:
# 多对多的合并，有些不直观

df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df1

key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5

In [78]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                   'data2': range(5)})

df2

key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4

In [66]:
pd.merge(df1, df2, on='key', how='left')

key  data1  data2
0    b      0    1.0
1    b      0    3.0
2    b      1    1.0
3    b      1    3.0
4    a      2    0.0
5    a      2    2.0
6    c      3    NaN
7    a      4    0.0
8    a      4    2.0
9    b      5    1.0
10   b      5    3.0

In [79]:
# 【多对多】连接产生的行是笛卡尔积：
# 由于左边的DataFrame有3个"b"行，右边的有2个，所以最终结果中有6个"b"行
# 连接方式只影响出现在结果中的不同的键的值

pd.merge(df1, df2, how='inner')

key  data1  data2
0   b      0      1
1   b      0      3
2   b      1      1
3   b      1      3
4   b      5      1
5   b      5      3
6   a      2      0
7   a      2      2
8   a      4      0
9   a      4      2

In [80]:
# 要根据多个键进行合并，传入一个由列名组成的列表即可

left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                      'key2': ['one', 'two', 'one'],
                      'lval': [1, 2, 3]})
left

key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3

In [81]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

right

key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7

In [82]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

key1 key2  lval  rval
0  foo  one   1.0   4.0
1  foo  one   1.0   5.0
2  foo  two   2.0   NaN
3  bar  one   3.0   6.0
4  bar  two   NaN   7.0

In [83]:
# 对于重复列名的处理
# merge的suffixes选项，用于指定附加到左右两个DataFrame对象的重叠列名上的字符串

pd.merge(left, right, on='key1')

key1 key2_x  lval key2_y  rval
0  foo    one     1    one     4
1  foo    one     1    one     5
2  foo    two     2    one     4
3  foo    two     2    one     5
4  bar    one     3    one     6
5  bar    one     3    two     7

In [84]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

key1 key2_left  lval key2_right  rval
0  foo       one     1        one     4
1  foo       one     1        one     5
2  foo       two     2        one     4
3  foo       two     2        one     5
4  bar       one     3        one     6
5  bar       one     3        two     7

In [85]:
%%html
<img src="picture/表8-2 merge函数的参数.png", width=600>
<img src="picture/表8-2 merge函数的参数(1).png", width=600>

## 8.2.2 根据索引合并

In [3]:
# 有时候，DataFrame中的连接键位于其索引中。
# 传入left_index=True or right_index=True（或两个都传）：说明索引应该被用作连接键

left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
left1

key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5

In [4]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

right1

group_val
a        3.5
b        7.0

In [6]:
pd.merge(left1, right1, left_on='key', right_index=True)

key  value  group_val
0   a      0        3.5
2   a      2        3.5
3   a      3        3.5
1   b      1        7.0
4   b      4        7.0

In [89]:
# 由于默认的merge方法是求取连接键的【交集】
# 因此可以通过【外连接】的方式得到【并集】

pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

key  value  group_val
0   a      0        3.5
2   a      2        3.5
3   a      3        3.5
1   b      1        7.0
4   b      4        7.0
5   c      5        NaN

In [90]:
# 对于层次化索引的数据，索引的合并默认是多键合并

lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})

lefth

key1  key2  data
0    Ohio  2000   0.0
1    Ohio  2001   1.0
2    Ohio  2002   2.0
3  Nevada  2001   3.0
4  Nevada  2002   4.0

In [91]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [92]:
# 在这种情况下，必须以列表的形式指明用作合并键的多个列（注意用how='outer'对重复索引值的处理

pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

key1  key2  data  event1  event2
0    Ohio  2000   0.0       4       5
0    Ohio  2000   0.0       6       7
1    Ohio  2001   1.0       8       9
2    Ohio  2002   2.0      10      11
3  Nevada  2001   3.0       0       1

In [93]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

key1  key2  data  event1  event2
0    Ohio  2000   0.0     4.0     5.0
0    Ohio  2000   0.0     6.0     7.0
1    Ohio  2001   1.0     8.0     9.0
2    Ohio  2002   2.0    10.0    11.0
3  Nevada  2001   3.0     0.0     1.0
4  Nevada  2002   4.0     NaN     NaN
4  Nevada  2000   NaN     2.0     3.0

In [94]:
# 同时使用合并双方的索引也没问题

left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                      index=['a', 'c', 'e'],
                      columns=['Ohio', 'Nevada'])

left2

Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0

In [95]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

right2

Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0

In [96]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

Ohio  Nevada  Missouri  Alabama
a   1.0     2.0       NaN      NaN
b   NaN     NaN       7.0      8.0
c   3.0     4.0       9.0     10.0
d   NaN     NaN      11.0     12.0
e   5.0     6.0      13.0     14.0

In [128]:
# DataFrame_instance.join(): 能更为方便地实现按索引合并;
# 还可用于合并多个带有相同或相似索引的DataFrame对象，但要求没有重叠的列。
# join方法，默认使用【左连接】，保留左边表的行索引。

left2.join(right2, how='outer')

Ohio  Nevada  Missouri  Alabama
a   1.0     2.0       NaN      NaN
b   NaN     NaN       7.0      8.0
c   3.0     4.0       9.0     10.0
d   NaN     NaN      11.0     12.0
e   5.0     6.0      13.0     14.0

In [129]:
# DataFrame_instance.join(): 还支持在调用的DataFrame的列上，连接传递的DataFrame索引

left1.join(right1, on='key')

key  value  group_val
0   a      0        3.5
1   b      1        7.0
2   a      2        3.5
3   a      3        3.5
4   b      4        7.0
5   c      5        NaN

In [136]:
# 对于简单的索引合并，还可以向join()传入一组DataFrame

another = pd.DataFrame([[7., 8., 9.], [9., 10., 11.], [11., 12., 13.], [16., 17., 18.]],
                      index=['a', 'c', 'e', 'f'],
                      columns=['New York', 'Oregon', 'Texas'])

In [137]:
another

New York  Oregon  Texas
a       7.0     8.0    9.0
c       9.0    10.0   11.0
e      11.0    12.0   13.0
f      16.0    17.0   18.0

In [144]:
right2

Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0

In [146]:
left2

Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0

In [147]:
left2.join([right2, another])

Ohio  Nevada  Missouri  Alabama  New York  Oregon  Texas
a   1.0     2.0       NaN      NaN       7.0     8.0    9.0
c   3.0     4.0       9.0     10.0       9.0    10.0   11.0
e   5.0     6.0      13.0     14.0      11.0    12.0   13.0

In [155]:
left2.join([right2, another],how='outer', sort=False)

D:\Users\HS\Anaconda3\lib\site-packages\pandas\core\frame.py:6369: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


Ohio  Nevada  Missouri  Alabama  New York  Oregon  Texas
a   1.0     2.0       NaN      NaN       7.0     8.0    9.0
b   NaN     NaN       7.0      8.0       NaN     NaN    NaN
c   3.0     4.0       9.0     10.0       9.0    10.0   11.0
d   NaN     NaN      11.0     12.0       NaN     NaN    NaN
e   5.0     6.0      13.0     14.0      11.0    12.0   13.0
f   NaN     NaN       NaN      NaN      16.0    17.0   18.0

## 8.2.3 轴向连接

In [3]:
import numpy as np
import pandas as pd

In [159]:
# 另一种【数据合并】运算也被称作连接(concatenation)、绑定(binding)或堆叠(stacking)
# numpy.concatenate()

arr = np.arange(12).reshape((3, 4))

In [160]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [161]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [162]:
# pandas.concat()

s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [164]:
# 对这些对象调用conca()函数，可以将【值】和【索引】【粘合】在一起

pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [170]:
# 默认情况下，concat()是在axis=0上工作的，最终产生一个新的Series。
# 如果传入axis=1,则结果就会变成一个DataFrame
# 这种情况下，另外的轴上没有重叠，从索引的有序并集（外连接）上就可以看出来。

pd.concat([s1, s2, s3], axis=1, sort=True)

0    1    2
a  0.0  NaN  NaN
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0

In [180]:
# 传入join='inner'即可得到它们的【交集】
# f, g 标签将会消失

s4 = pd.concat([s1, s3])

In [181]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [182]:
pd.concat([s1, s4], axis=1, sort=True)

0  1
a  0.0  0
b  1.0  1
f  NaN  5
g  NaN  6

In [183]:
pd.concat([s1, s4], axis=1, join='inner')

0  1
a  0  0
b  1  1

In [185]:
# 可以通过join_axes选项，指定要在其它轴上使用的索引

pd.concat([s1, s4], axis=1, join_axes=[['a', 'f', 'b', 'g']])

0  1
a  0.0  0
f  NaN  5
b  1.0  1
g  NaN  6

In [197]:
# 使用keys参数，可在想要的连接轴上创建一个层次化索引

result = pd.concat([s1,s1, s3], keys=['one', 'two', 'three'], sort=True)

In [195]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [196]:
result.unstack()

a    b    f    g
one    0.0  1.0  NaN  NaN
two    0.0  1.0  NaN  NaN
three  NaN  NaN  5.0  6.0

In [200]:
# 如果沿着axis=1对Series进行合并，则keys就会成为DataFrame的列头

pd.concat([s1,s2, s3],axis=1, keys=['one', 'two', 'three'], sort=True)

one  two  three
a  0.0  NaN    NaN
b  1.0  NaN    NaN
c  NaN  2.0    NaN
d  NaN  3.0    NaN
e  NaN  4.0    NaN
f  NaN  NaN    5.0
g  NaN  NaN    6.0

In [202]:
# 同样的逻辑也适用于DataFrame对象

df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])

df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

In [203]:
df1

one  two
a    0    1
b    2    3
c    4    5

In [204]:
df2

three  four
a      5     6
c      7     8

In [207]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], sort=True)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [210]:
# 传入字典，则字典的键就会被当做keys选项的值

pd.concat({'level1': df1, 'level2': df2}, axis=1, sort=True)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [211]:
# 可以用names参数命名创建的轴的级别

pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], 
          names=['upper', 'lower'], sort=True)

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [4]:
# DataFrame的【行索引】不包含任何相关数据

df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [5]:
df1

a         b         c         d
0  0.298996  0.966891  0.156301  0.551923
1  1.826793 -0.518390  0.489146 -1.538946
2 -0.625092  0.300367  1.414670  0.498582

In [6]:
df2

b         d         a
0 -0.109685  1.122846  1.346810
1 -0.619003  0.033174  0.846828

In [8]:
pd.concat([df1, df2], sort=True)

a         b         c         d
0  0.298996  0.966891  0.156301  0.551923
1  1.826793 -0.518390  0.489146 -1.538946
2 -0.625092  0.300367  1.414670  0.498582
0  1.346810 -0.109685       NaN  1.122846
1  0.846828 -0.619003       NaN  0.033174

In [217]:
# 传入ignore_index=True: 不保留连接轴上的索引，产生一组新索引 range(total_length)

pd.concat([df1, df2], ignore_index=True, sort=True)

a         b         c         d
0 -0.748532 -1.056169 -1.813277  0.976146
1 -0.208210  0.719616  0.179914  1.676682
2 -0.845262  0.397377 -0.762312  0.043125
3  0.211810  0.553991       NaN  0.413082
4  0.221622 -0.613546       NaN  0.814817

In [76]:
%%html
<img src="picture/8-3 concat函数的参数.png", width=600>

## 8.2.4 合并重叠数据

In [9]:
import numpy as np
import pandas as pd

In [10]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
               index=['f', 'e', 'd', 'c', 'b', 'a'])

b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])

In [11]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [12]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [13]:
b[-1] = np.nan

In [14]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [227]:
# np.where(): 表示一种等价于面向数组的if-else

np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [231]:
# Series_instance.first(): 实现的也是一样的功能，还带有pandas的数据对齐 

b[ : -2].combine_first(a[2: ])  # 类似于np.where()的用法

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [236]:
# 对于DataFrame，combine_first()可看做：用【传递对象】中的数据为【调用对象】的缺失数据“打补丁”

df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})

df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [234]:
df1

a    b   c
0  1.0  NaN   2
1  NaN  2.0   6
2  5.0  NaN  10
3  NaN  6.0  14

In [235]:
df2

a    b
0  5.0  NaN
1  4.0  3.0
2  NaN  4.0
3  3.0  6.0
4  7.0  8.0

In [237]:
# 用传递对象中的数据为调用对象的缺失数据“打补丁”

df1.combine_first(df2)

a    b     c
0  1.0  NaN   2.0
1  4.0  2.0   6.0
2  5.0  4.0  10.0
3  3.0  6.0  14.0
4  7.0  8.0   NaN

# 8.3 重塑和轴向旋转

## 8.3.1 重塑层次化索引

In [18]:
import numpy as np
import pandas as pd

In [19]:
# 层次化索引为DataFrame数据的重排任务提供了一种具有良好一致性的方式。
# stack: 将数据的【列】”旋转”为【行】
# unstack: 将数据的【行】”旋转”为【列】

data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                     index=pd.Index(['Ohio','Colorado'], name='state'),
                     columns=pd.Index(['one', 'two', 'three'],
                     name='number'))

data

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5

In [22]:
# 使用stack()方法

result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [23]:
# unstack()方法可将其重排为一个DataFrame

result.unstack()

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5

In [25]:
# 默认情况下，unstack()操作的是最内层（stack也是如此）。
# 传入分层级别【编号】或【名称】即可对其它级别进行unstack操作

result.unstack(0)

state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5

In [26]:
result.unstack('state')

state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5

In [27]:
# 如果不是所有的级别值都能在各分组中找到的话，则unstack操作可能会引入缺失数据

s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [28]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [29]:
# 引入缺失数据

data2.unstack()

a    b    c    d    e
one  0.0  1.0  2.0  3.0  NaN
two  NaN  NaN  4.0  5.0  6.0

In [30]:
# stack默认会滤除缺失数据
# 因此，该运算是可逆的

data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [31]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [32]:
# 在对DataFrame进行unstack操作时，作为【旋转轴】的级别将会成为结果中的【最低级别】

df = pd.DataFrame({'left': result, 'right': result + 5},
                   columns=pd.Index(['left', 'right'], name='side'))

df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [33]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [34]:
# 当调用stack，可以指明轴的名字

df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## 8.3.2 将“长格式”旋转为“宽格式”

In [35]:
# 多个时间序列数据通常是以“长格式”(long)或“堆叠格式”(stacked)存储在数据库和CSV中的

data = pd.read_csv('examples/macrodata.csv')

In [38]:
data.head()

year  quarter   realgdp  realcons  realinv  realgovt  realdpi    cpi  \
0  1959.0      1.0  2710.349    1707.4  286.898   470.045   1886.9  28.98   
1  1959.0      2.0  2778.801    1733.7  310.859   481.301   1919.7  29.15   
2  1959.0      3.0  2775.488    1751.8  289.226   491.260   1916.4  29.35   
3  1959.0      4.0  2785.204    1753.7  299.356   484.052   1931.3  29.37   
4  1960.0      1.0  2847.699    1770.5  331.722   462.199   1955.5  29.54   

      m1  tbilrate  unemp      pop  infl  realint  
0  139.7      2.82    5.8  177.146  0.00     0.00  
1  141.7      3.08    5.1  177.830  2.34     0.74  
2  140.5      3.82    5.3  178.657  2.74     1.09  
3  140.0      4.33    5.6  179.386  0.27     4.06  
4  139.6      3.50    5.2  180.007  2.31     1.19

In [40]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [42]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [44]:
data = data.reindex(columns=columns)
data.head()

item   realgdp  infl  unemp
0     2710.349  0.00    5.8
1     2778.801  2.34    5.1
2     2775.488  2.74    5.3
3     2785.204  0.27    5.6
4     2847.699  2.31    5.2

In [46]:
data.index = periods.to_timestamp('D', 'end')
data.head()

item                            realgdp  infl  unemp
date                                                
1959-03-31 23:59:59.999999999  2710.349  0.00    5.8
1959-06-30 23:59:59.999999999  2778.801  2.34    5.1
1959-09-30 23:59:59.999999999  2775.488  2.74    5.3
1959-12-31 23:59:59.999999999  2785.204  0.27    5.6
1960-03-31 23:59:59.999999999  2847.699  2.31    5.2

In [48]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata.head()

date     item     value
0 1959-03-31 23:59:59.999999999  realgdp  2710.349
1 1959-03-31 23:59:59.999999999     infl     0.000
2 1959-03-31 23:59:59.999999999    unemp     5.800
3 1959-06-30 23:59:59.999999999  realgdp  2778.801
4 1959-06-30 23:59:59.999999999     infl     2.340

In [49]:
# DataFrame_instance.pivot()完全可以实现这个转换

pivoted = ldata.pivot('date', 'item', 'value')  

# 前两个值分别用作【行】和【列】索引，最后一个可选值用于填充DataFrame的数据列

In [50]:
pivoted.head()

item                           infl   realgdp  unemp
date                                                
1959-03-31 23:59:59.999999999  0.00  2710.349    5.8
1959-06-30 23:59:59.999999999  2.34  2778.801    5.1
1959-09-30 23:59:59.999999999  2.74  2775.488    5.3
1959-12-31 23:59:59.999999999  0.27  2785.204    5.6
1960-03-31 23:59:59.999999999  2.31  2847.699    5.2

In [51]:
# 假设有两个需要同时【重塑】的数据列

ldata['value2'] = np.random.randn(len(ldata))

In [52]:
ldata[ : 10]

date     item     value    value2
0 1959-03-31 23:59:59.999999999  realgdp  2710.349 -1.438086
1 1959-03-31 23:59:59.999999999     infl     0.000 -0.334376
2 1959-03-31 23:59:59.999999999    unemp     5.800  1.651818
3 1959-06-30 23:59:59.999999999  realgdp  2778.801  1.545745
4 1959-06-30 23:59:59.999999999     infl     2.340 -0.314002
5 1959-06-30 23:59:59.999999999    unemp     5.100  2.062792
6 1959-09-30 23:59:59.999999999  realgdp  2775.488  0.490953
7 1959-09-30 23:59:59.999999999     infl     2.740  1.046299
8 1959-09-30 23:59:59.999999999    unemp     5.300 -0.708181
9 1959-12-31 23:59:59.999999999  realgdp  2785.204  0.488847

In [53]:
# 如果忽略最后一个参数，得到的DataFrame就会带有层次化的列

pivoted =ldata.pivot('date', 'item')

In [54]:
pivoted[ : 5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.334376 -1.438086   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.314002  1.545745   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  1.046299  0.490953   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.118671  0.488847   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.694696  0.677442   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.651818  
1959-06-30 23:59:59.999999999  2.062792  
1959-09-30 23:59:59.999999999 -0.708181  
1959-12-31 23:59:59.999999999  1.379093  
1960-03-31 23:59:59.999999999  1.315621

In [55]:
pivoted['value'][ : 5]

item                           infl   realgdp  unemp
date                                                
1959-03-31 23:59:59.999999999  0.00  2710.349    5.8
1959-06-30 23:59:59.999999999  2.34  2778.801    5.1
1959-09-30 23:59:59.999999999  2.74  2775.488    5.3
1959-12-31 23:59:59.999999999  0.27  2785.204    5.6
1960-03-31 23:59:59.999999999  2.31  2847.699    5.2

In [56]:
# 注意：pivot其实就是用set_index创建层次化索引，再用unstack重塑

unstacked = ldata.set_index(['date', 'item']).unstack('item')

In [57]:
unstacked[ : 7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.334376 -1.438086   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.314002  1.545745   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  1.046299  0.490953   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.118671  0.488847   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.694696  0.677442   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -1.193745 -0.192567   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -0.088901 -0.349833   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.651818  
1959-06-30 23:59:59.999999999  2.062792  
1959-09-30 23:59:59.999999999 -0.708181  
1959-12-31 23:59:59.999999999  1.379093  
1960-03-31 23:59:59.999999999  1.315621  
1960-06-30 23:59:59.999999999  0.764173  
1960-09-30 23:59:59.999999999  0.004621

## 8.3.3 将“宽格式”旋转为“长格式”

In [58]:
# 旋转DataFrame的逆运算是pandas.melt, 它合并多个列成为一个，产生一个比输入长的DataFrame

df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                    'A': [1, 2, 3],
                    'B': [4, 5, 6],
                    'C': [7, 8, 9]})

In [59]:
df

key  A  B  C
0  foo  1  4  7
1  bar  2  5  8
2  baz  3  6  9

In [60]:
# key列可能是分组指标，其它的列是数据值
# 当使用pandas.melt,必须指明哪些是分组指标

melted = pd.melt(df, ['key'])
melted

key variable  value
0  foo        A      1
1  bar        A      2
2  baz        A      3
3  foo        B      4
4  bar        B      5
5  baz        B      6
6  foo        C      7
7  bar        C      8
8  baz        C      9

In [293]:
# 使用pivot,可以重塑回原来的样子

reshaped = melted.pivot('key', 'variable', 'value')

In [294]:
reshaped

variable  A  B  C
key              
bar       2  5  8
baz       3  6  9
foo       1  4  7

In [295]:
# 因为pivot的结果从列创建了一个索引，用作行标签。
# 可以使用reset_index将数据移回列

reshaped.reset_index()

variable  key  A  B  C
0         bar  2  5  8
1         baz  3  6  9
2         foo  1  4  7

In [296]:
# 可以指定【列】的子集，作为值的列

pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

key variable  value
0  foo        A      1
1  bar        A      2
2  baz        A      3
3  foo        B      4
4  bar        B      5
5  baz        B      6

In [297]:
# pandas.melt()也可以不用分组指标

pd.melt(df, value_vars=['A', 'B', 'C'])

variable  value
0        A      1
1        A      2
2        A      3
3        B      4
4        B      5
5        B      6
6        C      7
7        C      8
8        C      9

In [298]:
pd.melt0(df, value_vars=['key', 'A', 'B'])

variable value
0      key   foo
1      key   bar
2      key   baz
3        A     1
4        A     2
5        A     3
6        B     4
7        B     5
8        B     6